In [4]:
# pip install urllib.request  # restart kernel

import urllib.request

url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)


('the-verdict.txt', <http.client.HTTPMessage at 0x1c1ff9c4440>)